<a href="https://colab.research.google.com/github/soumabkargit/Extract_Text_Image/blob/master/TesseractMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extraction du montant ttc sur les photos de facture**

#installation de l'ocr Tesseract, prise en charge de Tesseract en Anglais et installation de remotezip. 

[https://github.com/tesseract-ocr/tesseract/wiki](https://)


In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-eng
!pip install remotezip
!pip install -q -U opencv-python

##Importation de toutes les dépendances

In [ ]:
from matplotlib.image import imread
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
import pytesseract
from pytesseract import Output
import matplotlib.patches as patches
from PIL import Image
import re
from pathlib import Path
from shutil import copyfile
from remotezip import RemoteZip
import json
import csv
# Technically not necessary in newest versions of jupyter
%matplotlib inline

## Déclarations et assignations de toutes les variables globales

In [ ]:
url = 'https://expressexpense.com/large-receipt-image-dataset-SRD.zip'

lang = 'eng'

image_extract_path = "image_extract"
image_NO_extract_path = "image_NO_extract"
image_NO_read_text_path = "image_NO_read_text"
data = "data"

file_info_result = {}

list_regex = []
regex_1 = r'TOTAL(.)*(\:)*(.)*(\d)'
regex_2 = r'BALANCE(.)*(\:)*(.)*(\d)'
regex_3 = r'AMOUNT(.)*(\:)*(.)*(\d)'
regex_4 = r'GRAND(.)*TOTAL(.)*(\:)*(.)*(\d)'
regex_5 = r'ORDER(.)*BALANCE(.)*(\:)*(.)*(\d)'
regex_6 = r'CASH(.)*(\:)*(.)*(\d)'
regex_amount = r'\d*\.\d+|\d+'

list_regex.append(regex_1)
list_regex.append(regex_2)
list_regex.append(regex_3)
list_regex.append(regex_4)
list_regex.append(regex_5)
list_regex.append(regex_6)

good_info_candidat = 'good_info_candidat'
bad_info_candidat_value = 'bad_info_candidat_value'
bad_info_candidat_no_value = 'bad_info_candidat_no_value'
bad_info_not_foreseen = 'bad_info_not_foreseen'
warning_situation = 'warning_situation'

all_info_csv_file = "all_info.csv"
all_info_bad_csv_file = "all_info_bad.csv"
all_info_good_csv_file = "all_info_good.csv"

good_state = 'Good_State'
bad_state = 'Bad_State'

##Implémentation de toutes les méthodes utiles pour l'extraction du montant TTC

In [ ]:
def hasNumbers(inputString):
  """Vérifier si la chaîne est alphanumérique."""
  return any(char.isdigit() for char in inputString)

def add_regex(list_regex, regex):
  """Ajouter une expression régulière."""
  list_regex.append(regex)

def extract_amount(regex_amount, string):
  """Appliquer l'expression régulière pour avoir le montant TTC en chiffre."""
  return re.findall(regex_amount, string, re.IGNORECASE)

def createFolder(path):
  """Créer un dossier."""
  try:
    os.mkdir(path)
  except OSError:
    print ("la création du dossier %s a échoué" % path)
  else:
    print ("Réussite de la création du dossier %s " % path)

def extracted(url, data):
  """Téléchargement des données zipées et extraction dans un dossier"""
  try:
    with RemoteZip(url) as zip:
      zip.extractall(path=data)
  except :
    print ("Le zip n'a pas pu être chargé au lien %s " % url)
  
def is_all_values_table_null(array):
  test = False
  if all('' == s or s.isspace() for s in array):
    test = True
  return test

def showAmount(fonction):
  def wrapper(args):
      result = 0
      if len(args) == 1 :
        temp = args[0]
        result = float(temp)
      else:
        result = "degraded information"
      return fonction(result)
  return wrapper

@showAmount
def showAmountResut(amount):
  return amount
  

def extracted_text_split(imageName, lang):
  """Extraire le text sur l'image et spliter le text selon espace."""
  #im = np.array(Image.open(imageName), dtype=np.uint8) 
  im = cv2.imread(imageName)
  try :
    extracted_text = pytesseract.image_to_string(im, lang)
  except :
    #L image n'a pas pu être traité. C'est a dire l'extraction du texte a échoué.
    copy_file_folder(data + '/' + file_name, image_NO_extract_path + '/' + file_name)
  text_split_line = extracted_text.splitlines()
  return text_split_line

def extracted_text_split_file_name(file_path):
  """Faire l'extraction en précisant le chemen de l'image."""
  im = file_path
  text_split_text = extracted_text_split(im, lang)
  return text_split_text

def read_all_line_file(text_split_text, file_name):
  """Rechercher les information selon les patterns défis."""
  file_info_read_all_line_file = {}
  if is_all_values_table_null(text_split_text):
    copy_file_folder(data + '/' + file_name, image_NO_read_text_path + '/' + file_name)
    file_info_read_all_line_file[bad_info_candidat_no_value] = "Nous avons une chaîne vide, Verifions s'il n'y a aucun texte sur l'image."
    file_info_read_all_line_file[warning_situation] = "Erreur 002"
  for line in text_split_text:
    for regex in list_regex:
      if regexLine(regex, line):
        if hasNumbers(line):
          file_info_read_all_line_file[good_info_candidat] = showAmountResut(extract_amount(regex_amount, line))
        else:
          file_info_read_all_line_file[bad_info_candidat_value] = line
          file_info_read_all_line_file[warning_situation] = "Erreur 003"
  return file_info_read_all_line_file

def regexLine(regex, line):
  """Appliquer une expression régulière."""
  return re.search(regex, line, re.IGNORECASE)
    
def copy_file_folder(file_name, path):
  """Copier un fichier d'un dossier à un autre."""
  copyfile(file_name, path)

def get_all_names_files(data):
  """Avoir la liste de tout les fichiers."""
  try :
    list_files = os.listdir(data)
  except OSError:
    print ("La lecture de tout les fichiers dans le dossier %s a échoué" % data)
  else:
    print ("Réussite de la lecture de tout les fichiers dans le dossier %s " % data)
  return list_files

def get_all_amount(data):
  """Avoir le montant TTC de toute les photos de facture lues dans un dossier fournie."""
  files_infos_results = []
  temp_list_files = {}

  temp_list_files = get_all_names_files(data)

  if len(temp_list_files) == 0 :
    print("Le dossier est vide")
  else :
    for image_filename in temp_list_files:
      file_info_res = get_amount(image_filename)
      files_infos_results.append(file_info_res)
  return files_infos_results

def get_amount(file_name):
   """Avoir le montant TTC d'une photo de facture."""
   file_info_result = {}
   amount_info = {}
   text_split_text  = extracted_text_split_file_name(data + '/' + file_name)
   temp_file_info_result = read_all_line_file(text_split_text,file_name) 
   file_info_result["file_Name"] = file_name
   file_info_result['Amount'] = temp_file_info_result
   amount_info = file_info_result
   file_info_result = {}
   return amount_info

def create_csv(csv_file_name):
  """Ajouter une information au rapport cvs """
  # field names 
  fields = ['Amount', 'File_Name', 'File_State'] 
  # name of csv file 
  filename = csv_file_name
  # writing to csv file 
  f = open(filename, 'w')
  with f as csvfile: 
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames = fields)
    # writing headers (field names)   
    writer.writeheader()
    f.close()  

def save_data_csv(dict_data, csv_file):
  """Ajouter une information au rapport cvs """
  if isinstance(dict_data, list) == False :
    tab_res_info = []
    dict_data = tab_res_info.append(dict_data)
  dict_data = dict_data
  filename = csv_file
  fields = ['Amount', 'File_Name', 'File_State'] 
  f = open(filename, 'a')
  # writing to csv file 
  with f as csvfile: 
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames = fields)
    writer.writerows(dict_data)  
    f.close() 

def info_csv(file_name, value):
  """Construction de dictionnaire pour le rapport cvs """
  temp_csv_info_save = {}
  temp_csv_info_save['File_Name'] = file_name
  temp = value
  key_temp = list(temp.keys())
  if good_info_candidat in key_temp:
    temp_csv_info_save['Amount']= value[good_info_candidat]
    state = good_state
  elif ((bad_info_candidat_value in key_temp) ) : 
    state = bad_state 
    temp_csv_info_save['Amount']= value[bad_info_candidat_value]
  elif ( (bad_info_candidat_no_value in key_temp) ) :
    state = bad_state 
    temp_csv_info_save['Amount']= value[bad_info_candidat_no_value]
  elif ((bad_info_not_foreseen in key_temp) ) :
    state = bad_state 
    temp_csv_info_save['Amount']= value[bad_info_not_foreseen]
  else :
    state = bad_state   
    temp_csv_info_save['Amount']= "Erreur Non Prévuer"  
  temp_csv_info_save['File_State'] = state
  return temp_csv_info_save

def save_info(element):
  temp_csv_info_save = {}
  all_info_good_array = []
  all_info_bad_array = []
  all_info_array = []
  value_amount = element["Amount"]
  value_file = element["file_Name"]
  res_key = list(value_amount.keys())
  if good_info_candidat in res_key:
    copyfile(data + '/' + value_file, image_extract_path + '/' + value_file)
    temp_csv_info_save = info_csv(value_file, value_amount )
    all_info_good_array.append(temp_csv_info_save)
    all_info_array.append(temp_csv_info_save)
  elif ( (bad_info_candidat_value in res_key) ) : 
    copyfile(data + '/' + value_file, bad_info_candidat_value + '/' + value_file)
    temp_csv_info_save = info_csv(value_file, value_amount)
    all_info_bad_array.append(temp_csv_info_save)
    all_info_array.append(temp_csv_info_save)
  elif ((bad_info_not_foreseen in res_key) ) :
    copyfile(data + '/' + value_file, bad_info_candidat_no_value + '/' + value_file)
    temp_csv_info_save = info_csv(value_file, value_amount )
    all_info_bad_array.append(temp_csv_info_save)
    all_info_array.append(temp_csv_info_save)
  elif ((bad_info_not_foreseen in res_key) ) :
    copyfile(data + '/' + value_file, bad_info_not_foreseen + '/' + value_file)
    temp_csv_info_save = info_csv(value_file, value_amount )
    all_info_bad_array.append(temp_csv_info_save)
    all_info_array.append(temp_csv_info_save)
  else : 
    copyfile(data + '/' + value_file, warning_situation + '/' + value_file)
    temp_csv_info_save = info_csv(value_file, value_amount )
    all_info_bad_array.append(temp_csv_info_save)
    all_info_array.append(temp_csv_info_save)
  save_data_csv(all_info_bad_array, all_info_bad_csv_file)
  save_data_csv(all_info_array, all_info_csv_file)
  save_data_csv(all_info_good_array, all_info_good_csv_file)
     
def save_all_infos(file_info_res):
  """Ranger la facture selon le résultat du traitement."""
  if isinstance(file_info_res, list) == False :
    save_info(file_info_res)
  for element in file_info_res :
    save_info(element)


##Création des dossiers
Dossier de toute les photos de facture

Dossier des factures dont l'extraction du montant ttc a réussi

Dossier des factures dont l'extraction des montant ttc a échoué


In [ ]:
createFolder(image_extract_path)
createFolder(image_NO_extract_path)
createFolder(image_NO_read_text_path)
createFolder(bad_info_candidat_no_value)
createFolder(bad_info_candidat_value)
createFolder(bad_info_not_foreseen)
createFolder(data)
createFolder(warning_situation)

In [ ]:
create_csv(all_info_csv_file)
create_csv(all_info_bad_csv_file)
create_csv(all_info_good_csv_file)

Téléchargement du zip des photos et extraction dans le dossier de toute les factures

In [ ]:
extracted(url, data)

Extraction de montant ttc pour une facture

In [ ]:
file_name = '1008-receipt.jpg'
res = get_amount(file_name)
print("Le nom du fichier est : " + res['file_Name']+ "\n" + "et son montant est : " ,res['Amount'] )

In [ ]:
save_info(res)

Extraction de montant ttc pour toute les factures

In [ ]:
dict_dat = get_all_amount(data)

Réussite de la lecture de tout les fichiers dans le dossier data 


In [ ]:
dict_dat

In [ ]:
save_all_infos(dict_dat)

#References

https://nanonets.com/blog/receipt-ocr/?fbclid=IwAR0H4VkHhpimqKIgEUQOUCOJS1mtk6vQREcv3MMiqJ2NpxG9LHZnsHM-aeE


https://www.datacamp.com/community/tutorials/zip-file